# Directive for creating a script for your notebook

The block here below is required at the top of each notebook that you want to create a script for. You will also need to edit the "settings.ini" file, to create a script (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details). Replace **some_string** with a name that makes sense for your notebook. 

In [1]:
#| default_exp Ecoli_parser

# Libraries
Include all the libraries which should be used in this *Escherichia coli* module, to create log the various operations to load input files, craete datastructures, maniplate and output desired results.

In [2]:
#| export

import os
import sys
import pandas as pd 
from pathlib import Path
import logging
from datetime import datetime 
from typing import List, Dict 
from fastcore.script import call_parse
#import functions from core module (optional, but most likely needed). 
from ssi_analysis_result_parsers import(
    core
)



In [3]:
# This block should never be exported. It is to have python running in the project (and not the nbs) dir, and to initiate the package using pip.
os.chdir(core.PROJECT_DIR)

# Functions
Necessary functins to extract *k-mer alignment (KMA tool)* results from *.res* files to perform following steps

- Extract data 
- Filter data based on gene-specific thresholds for the *template coverage* and *Query identity*
- Perform *OH* typing and *stx* detection based on gene-specific information
- Wrangle data using *pandas* dataframes creating accurate data structure
- Store results in either sample-specific output files or one full tsv file extending the original samplesheet information

In [4]:
#| export

thresholds = {
    "stx": [98, 98],
    "wzx": [98, 98],
    "wzy": [98, 98],
    "wzt": [98, 98],
    "wzm": [98, 98],
    "fliC": [90, 90],
    "fli": [90, 90],
    "eae": [95, 95],
    "ehxA": [95, 95],
    "other": [98, 98]
}

### Thresholds

Defines gene-specific tresholds used to filter the input *KMA .res* files on the columns [*template coverage*,*Query identity*] to identify genes related to:
- Shiga Toxin (stx1 and stx2 subtypes)
- Serotyping using O-antigens with Serotype genes (wzx,wzy,wzt,wzm) and H-antigen using Flagellar genes (fli & fliC)
- Surface adhesion for EPEC and STEC strains with eae gene 
- Hemolytic toxin (ehxA)

### Logging

Defines a logging setup function used throughout the later functions to record information and errors both to a per-sample log file. 

The log file is created inside the specified log directory with a filename based on the sample name. The log file contains detailed messages with information of the progression, errors and warnings with timestamps. Which is useful for tracing execution and diagnosing issues per sample in multi-sample pipelines.

In [5]:
#| export

def setup_logging(log_dir: str, sample_name: str) -> None:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"{sample_name}_kma_fbi.log")

    logger = logging.getLogger()
    while logger.hasHandlers():
        logger.removeHandler(logger.handlers[0])

    logging.basicConfig(
        filename=log_file,
        filemode="a",
        format="%(asctime)s - %(levelname)s - %(message)s",
        level=logging.INFO
    )

    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(console_handler)

    logging.info(f"Logging started for {log_file}")


### Thresholds filtering & .res parsing

For each gene specified earlier, the thresholds for the template coverage and query identity is extracted and used to filter the input *KMA .res* file

In [6]:
#| export

def get_threshold(template_name: str, thresholds: Dict[str, List[int]]) -> List[int]:
    """
    Returns the coverage and identity threshold for a given gene.

    Args:
        template_name (str): Name of the template (gene) from the .res file.
        thresholds (Dict[str, List[int]]): Dictionary of gene thresholds.

    Returns:
        List[int]: A list of two integers: [coverage_threshold, identity_threshold].
    """
    for key in thresholds:
        if key in template_name:
            return thresholds[key]
    return thresholds["other"]

def process_res_file(res_file_path: str) -> pd.DataFrame:
    """
    Reads and filters a KMA .res file based on predefined thresholds.

    Args:
        res_file_path (str): Path to the .res file.
        thresholds (Dict[str, List[int]]): Gene-specific thresholds.

    Returns:
        pd.DataFrame: Filtered results DataFrame.
    """
    try:
        res_df = pd.read_csv(res_file_path, sep="\t")
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {res_file_path}")
    except pd.errors.EmptyDataError:
        raise ValueError(f"File is empty or not properly formatted: {res_file_path}")

    required_columns = {"#Template", "Template_Coverage", "Query_Identity", "Depth"}
    if not required_columns.issubset(res_df.columns):
        raise ValueError(f"Missing expected columns in {res_file_path}")

    res_df["threshold"] = res_df["#Template"].apply(lambda x: get_threshold(x, thresholds))
    res_df_filtered = res_df[
        (res_df["Template_Coverage"] >= res_df["threshold"].apply(lambda x: x[0])) &
        (res_df["Query_Identity"] >= res_df["threshold"].apply(lambda x: x[1]))
    ]
    return res_df_filtered


### Escherichia coli results

Defines a class to parse, summarize and export the *E. coli* gene typing results from the filtered .res input files. 
It creates per-sample logging and outputs the results either pr sample in their respective directories with all input files or as one conjoined file extending the input samplesheet file. The class allows for an additional column in the final output with a more detailed and summarize gene-specific information of the Depth,template coverage, query identity.

With the final output with verbose option having the following structure

| sample_name | illumina_read_files | nanopore_read_file | assembly_file | organism | variant | notes | stx | OH | wzx | wzy | wzt | wzm | eae | ehxA | Other | verbose|
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |

The Serotyping results in the above structure has gene-specific requirements.

- If *wzx* and *wzy* are the sole O-antigen genes with no conflicts between them, that information is used as the O-type result and stored in the *OH* column, otherwise the results are store in the *wzx* and *wzy* columns and O-type in the *OH* column remains empty 
- If *wzt* and *wzm* are the sole O-antigen genes with no conflicts between them, that information is used as the O-type result and stored in the *OH* column, otherwise the results are store in the *wzt* and *wzm* columns and O-type in the *OH* column remains empty  
- If all O-antigen genes exist in the *KMA* results, the O-type in the *OH* column remains empty and all four *wzx,wzy,wzt,wzm* columns are filled
- If *fli* exist in the *KMA* results that information is used as the H-type and stored in the *OH* column. The only scenario where *fliC* is used to determine the H-type and stored in the *OH* column, is when *fli* doesn't exist.

For any potential conflict the user is referred to the additional information stored within the *verbose* column to accurately determine the serotypes

In [7]:
#| export

class EcoliResults:
    """
    Object for holding and processing E. coli typing results.

    This class stores summary typing data for multiple samples, provides utilities for per-sample processing, and export results in a tab-seperated format (.tsv).
    """

    # converts the sample results in dict to pandas df
    def __init__(self, results_dict: dict):
        """
        Initializes the EcoliResults object with typing result data.

        Args:
            results_dict (dict): Dictionary where keys are sample names and values are summary result dictionaries.
        """
        self.results_dict = results_dict
        self.results_df = pd.DataFrame.from_dict(results_dict, orient="index").reset_index(names="sample_name")

    @staticmethod
    def summarize_single_sample(sample_name: str, res_path: str, verbose_flag: int = 1) -> dict:
        """
        Processes a single sample KMA .res file and returns a summary dictionary.

        Args:
            sample_name (str): Sample identifier.
            res_path (str): Path to the sample's .res file.
            verbose_flag (int, optional): Include verbose info if set to 1. Default is 1.

        Returns:
            Dict[str, str]: Summary values extracted from the .res file.
        """
        log_dir = "examples/Log"
        setup_logging(log_dir, sample_name)

        NA_string = "-"
        output_data = {
            "stx": NA_string,
            "OH": NA_string, "wzx": NA_string, "wzy": NA_string, "wzt": NA_string, "wzm": NA_string,
            "eae": NA_string, "ehxA": NA_string,
            "Other": NA_string
        }

        try:
            logging.info(f"Processing .res file: {res_path}")
            filtered_df = process_res_file(res_path)
        except Exception as e:
            logging.error(f"Failed to process {res_path}: {e}")
            return output_data

        gene_map = {
            "wzx": "wzx", "wzy": "wzy", "wzt": "wzt", "wzm": "wzm",
            "eae": "eae", "ehxA": "ehxA"
        }
        toxin = "stx"
        stx_alleles = set()
        fli = NA_string
        fliC = NA_string

        for template in filtered_df["#Template"]:
            parts = template.split("__")
            if len(parts) < 3:
                continue
            gene, allele = parts[1], parts[2]

            if gene in ["eae", "ehxA"]:
                output_data[gene] = "Positive"
            elif gene in gene_map:
                output_data[gene] = allele
            elif gene == "fliC":
                fliC = allele
            elif gene == "fli":
                fli = allele
            elif gene.startswith(toxin):
                stx_alleles.add(allele)
            elif gene not in thresholds:
                output_data["Other"] = allele

        if stx_alleles:
            output_data[toxin] = ";".join(sorted(stx_alleles))

        # serotype specific requirements
        wzx, wzy, wzt, wzm = output_data["wzx"], output_data["wzy"], output_data["wzt"], output_data["wzm"]
        Otype = "-"
        if wzx != NA_string and wzy != NA_string and wzx == wzy and wzt == NA_string and wzm == NA_string:
            Otype = wzx
            output_data["wzx"] = output_data["wzy"] = NA_string
        elif wzt != NA_string and wzm != NA_string and wzt == wzm and wzx == NA_string and wzy == NA_string:
            Otype = wzt
            output_data["wzt"] = output_data["wzm"] = NA_string

        Htype = fli if fli != NA_string else fliC
        output_data["OH"] = f"{Otype};{Htype}"

        # adding the additional depth, template coverage and query identity information
        if verbose_flag == 1:
            verbose_parts = []
            for _, row in filtered_df.iterrows():
                parts = row["#Template"].split("__")
                if len(parts) >= 3:
                    gene, allele = parts[1], parts[2]
                    depth = row["Depth"]
                    coverage = row["Template_Coverage"]
                    identity = row["Query_Identity"]
                    verbose_parts.append(f"{gene}_{allele}_{depth:.2f}_{coverage:.2f}_{identity:.2f}")
            output_data["verbose"] = ";".join(verbose_parts)

        logging.info(f"Successfully processed sample: {sample_name}")
        return output_data

    @classmethod
    def from_samplesheet(cls, 
                        samplesheet_path: Path, 
                        verbose: int = 1, 
                        results_base: str = "examples/Results/{sample_name}/kma/{sample_name}.res",
                    ) -> "EcoliResults":
        """
        Loads sample data from a samplesheet and summarizes each sample.

        Args:
            samplesheet_path (Path): Path to the samplesheet TSV file.
            verbose (int, optional): Whether to include verbose output per sample. Default is 1.

        Returns:
            EcoliResults: An instance of the class populated with summaries for all samples.
        """
        df = pd.read_csv(samplesheet_path, sep="\t")
        df.columns = df.columns.str.strip()
        #print("I AM INSIDE FROM SAMPLESHEET")
        #if "Illumina_read_files" in df.columns and ("read1" not in df.columns or "read2" not in df.columns):
        #    df[["read1", "read2"]] = df["Illumina_read_files"].str.split(",", expand=True)

        results_dict = {}
        for idx, row in df.iterrows():
            sample_name = row["sample_name"]
            res_path = Path(results_base.format(sample_name=sample_name)) #results_base / sample_name / "kma" / f"{sample_name}.res"
            #print(f"The res path is : {res_path}")
            summary = cls.summarize_single_sample(sample_name, res_path, verbose_flag=verbose)
            results_dict[sample_name] = summary
        
        # Convert to DataFrame
        result_df = pd.DataFrame.from_dict(results_dict, orient="index").reset_index(names="sample_name")

        # Merge with original metadata
        merged_df = df.merge(result_df, on="sample_name", how="left")

        # Create and return object
        obj = cls(results_dict)
        obj.results_df = merged_df
        return obj

    def write_tsv(self, output_file: Path):
        """
        Writes the summarized typing results to a TSV file.

        Args:
            output_file (Path): Destination file path for the output table.
        """
        self.results_df.to_csv(output_file, sep="\t", index=False)

    def __repr__(self):
        """
        Returns a concise summary of the results object.

        Returns:
            str: A string with sample and variable counts.
        """
        return f"<EcoliResults: {len(self.results_df)} samples, {len(self.results_df.columns)} variables>"

### Parser

Defines the parser of the samplesheet to utilize the previously defined and described functionality to summarizes E. coli typing results for each sample.
Outputs a TSV file or prints the results as a DataFrame, optionally with verbose detail.

In [8]:
#| export
@call_parse
def ecoli_parser(
    samplesheet_path: Path,  # Input samplesheet
    output_file: Path = None,  # Path to output
    verbose: int = 1,  # Verbosity,
    results_base: str = "examples/Results/{sample_name}/kma/{sample_name}.res"  # Path template for .res files
):
    results = EcoliResults.from_samplesheet(samplesheet_path, verbose=verbose, results_base=results_base)
    if output_file:
        results.write_tsv(output_file)
    else:
        print(results.results_df)


## TESTING - empirical dataset

Perform inline testing on two empirical datasets that verifies the functionality of the EcoliResults pipeline and datawrangling depending on the input



In [9]:
#| export
#| eval: true
import pandas as pd
from pathlib import Path
import os

# Define paths
samplesheet_path = Path("test_input/Ecoli/samplesheet.tsv")
output_dir = Path("test_output/Ecoli")

# Create output directory
if not output_dir.exists():
    output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "KMA_cases_parser.tsv"

# Assert input exists
assert samplesheet_path.exists(), f"File does not exist: {samplesheet_path}"
print(output_path)

# try the ecoli parser to see if the wrangling functionality works
try:
    ecoli_parser(
        samplesheet_path=samplesheet_path,
        output_file=output_path,
        verbose=1,
        results_base="test_input/Ecoli/{sample_name}.res"
    )
except Exception as e:
    raise AssertionError(f"Parser execution failed: {e}")

# compare the output with the expected results based on input to ensure correct wrangling

# read the created output files and check the information
sample_sheet_df = pd.read_csv(samplesheet_path, sep="\t")
sample_output_df = pd.read_csv(output_path, sep="\t")

### Test case 1. Check if the datastructure is correct
original_cols = sample_sheet_df.columns.tolist()
output_cols = sample_output_df.columns.tolist()
output_initial_cols = sample_output_df.columns[:len(original_cols)].tolist()
output_specific_cols = sample_output_df.columns[len(original_cols):].tolist()

assert original_cols == output_initial_cols, (
    f"Mismatch in first columns:\nExpected: {original_cols}\nGot: {output_initial_cols}"
)

assert output_specific_cols

### Test case 2. Check sample ERR3528110 which is correctly believed to be e.coli and ensure datawrangling does as expected
ERR3528110_res_path = "test_input/Ecoli/ERR3528110.res"
ERR3528110_input_df = pd.read_csv(ERR3528110_res_path, sep="\t")

ERR3528110_row = sample_output_df[sample_output_df["sample_name"] == "ERR3528110"].iloc[:,len(original_cols):len(output_cols)].iloc[0]

#extract the original genes from the res
gene_hits = ERR3528110_input_df["#Template"].tolist()

parsed_hits = []

for hit in gene_hits:
    parts = hit.split("__")
    assert len(parts) != 3, f"Unexpected KMA result format in: '{hit}'. Expected at least 3 '__' parts (e.g., ref__gene__allele) as off ecoli fbi 24-04-2025."
    gene, allele = parts[1], parts[2]
    parsed_hits.append((gene, allele))

# Extract OH genes 
O_gene_alleles = {gene: allele for gene, allele in parsed_hits if gene in {"wzx", "wzy", "wzt", "wzm"}}
H_gene_alleles = {gene: allele for gene, allele in parsed_hits if gene in {"fli", "fliC"}}

O_type = ERR3528110_row["OH"].split(";")[0]
H_type = ERR3528110_row["OH"].split(";")[1]

O_gene_keys = set(O_gene_alleles.keys())
H_gene_keys = set(H_gene_alleles.keys())

O_genes_no = len(O_gene_keys)
H_genes_no = len(H_gene_keys)

# O typing scenarios
# Case 1: wzx/wzy match
if O_gene_keys == {"wzx", "wzy"} and O_gene_alleles["wzx"] == O_gene_alleles["wzy"]:
    expected_otype = O_gene_alleles["wzx"]
    assert O_type == expected_otype, f"Expected OH '{expected_otype}', got '{O_type}'"
    # wzx/wzy should be suppressed
    assert ERR3528110_row["wzx"] == "-", "wzx column should be '-' when OH is used"
    assert ERR3528110_row["wzy"] == "-", "wzy column should be '-' when OH is used"
    #print(f"O-type correctly assigned from matching wzx/wzy: {O_type}")

# Case 2: wzt/wzm match
elif O_gene_keys == {"wzt", "wzm"} and O_gene_alleles["wzt"] == O_gene_alleles["wzm"]:
    expected_otype = O_gene_alleles["wzt"]
    assert O_type == expected_otype, f"Expected OH '{expected_otype}', got '{O_type}'"
    assert ERR3528110_row["wzt"] == "-", "wzt column should be '-' when OH is used"
    assert ERR3528110_row["wzm"] == "-", "wzm column should be '-' when OH is used"
    #print(f"O-type correctly assigned from matching wzt/wzm: {O_type}")

# Case 3: Conflict (≥3 genes, or 2 mismatched genes)
elif O_genes_no >= 3 or (
    (O_gene_keys == {"wzx", "wzy"} and O_gene_alleles["wzx"] != O_gene_alleles["wzy"]) or
    (O_gene_keys == {"wzt", "wzm"} and O_gene_alleles["wzt"] != O_gene_alleles["wzm"])
):
    assert O_type == "-", f"Expected OH = '-' due to conflict, got: '{O_type}'"
    for gene in O_gene_keys:
        assert ERR3528110_row[gene] == O_gene_alleles[gene], f"{gene} column should contain '{O_gene_alleles[gene]}'"
    #print("Conflict in O-typing correctly led to OH = '-' and individual gene columns retained.")

# H typing scenarios

# Case 1: If fli is present it will always take precedence over fliC
if H_gene_keys == {"fli"}:
    expected_htype = H_gene_alleles["fli"]
    assert H_type == expected_htype, f"Expected OH '{expected_htype}' from 'fli', got '{H_type}'"

# Case 2: only if fliC is the sole gene it is used
elif H_gene_keys == {"fliC"}:
    expected_htype = H_gene_alleles["fliC"]
    assert H_type == expected_htype, f"Expected OH '{expected_htype}' from 'fliC', got '{H_type}'"

# Case 3: if none exist the H type remains empty
else:
    assert H_type == "-", f"Expected H-type '-', but got '{H_type}'"

### Test case 3. Check sample ERR14229029 which is believed to be e.coli in the samplesheet is empty, as a result of being erroneously classified as e.coli

ERR14229029_row = sample_output_df[sample_output_df["sample_name"] == "ERR14229029"].iloc[:,len(original_cols):len(output_cols)].iloc[0]

ERR14229029_expected_values = ['-', '-;-', '-', '-', '-', '-', '-', '-', '-', float('nan')]
ERR14229029_values = [ERR14229029_row[col] for col in output_specific_cols]

for col, actual, expected in zip(output_specific_cols, ERR14229029_values, ERR14229029_expected_values):
    if pd.isna(expected):
        assert pd.isna(actual), f"{col}: Expected NaN, got {actual}"
    else:
        assert actual == expected, f"{col}: Expected '{expected}', got '{actual}'"


test_output/Ecoli/KMA_cases_parser.tsv
Logging started for examples/Log/ERR3528110_kma_fbi.log
Processing .res file: test_input/Ecoli/ERR3528110.res
Successfully processed sample: ERR3528110
Logging started for examples/Log/ERR14229029_kma_fbi.log
Processing .res file: test_input/Ecoli/ERR14229029.res
Successfully processed sample: ERR14229029


## TESTING - syntehtic data

Defines several scenarions of 12 samples representing syntehtic *KMA .res* content to perform several case specific results


In [10]:
#| export
import os
from tempfile import TemporaryDirectory
from pathlib import Path

test_cases = [
    # sample_name, res_content, expected_oh, expected_stx, expected_eae, expected_ehxA
    ("sample1", "1__wzx__O103__X\t100\t100\t60\n2__wzy__O103__X\t100\t100\t65\n3__fliC__H2__X\t100\t100\t70", "O103;H2", "-", "-", "-"),
    ("sample2", "1__wzt__O8__X\t100\t100\t60\n2__wzm__O8__X\t100\t100\t65\n3__fliC__H10__X\t100\t100\t70\n4__stx2__stx2-a__X\t100\t100\t90\n5__eae__eae-5__X\t100\t100\t80", "O8;H10", "stx2-a", "Positive", "-"),
    ("sample3", "1__fliC__H7__X\t100\t100\t70", "-;H7", "-", "-", "-"),
    ("sample4", "bad_line\n2__wzy__O111__X\t100\t100\t70\n3__fliC__H11__X\t100\t100\t70", "-;H11", "-", "-", "-"),
    ("sample5", "", "-;-", "-", "-", "-"),
    ("sample6", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O157__X\t100\t100\t65\n3__wzt__O8__X\t100\t100\t60\n4__wzm__O8__X\t100\t100\t65\n5__fli__H2__X\t100\t100\t70", "-;H2", "-", "-", "-"),
    ("sample7", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O111__X\t100\t100\t65\n3__fliC__H9__X\t100\t100\t70", "-;H9", "-", "-", "-"),
    ("sample8", "1__fli__H1__X\t100\t100\t70\n2__fliC__H12__X\t100\t100\t70", "-;H1", "-", "-", "-"),
    ("sample9", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O157__X\t100\t100\t65\n3__wzt__O8__X\t100\t100\t60\n4__wzm__O8__X\t100\t100\t65\n5__fliC__H10__X\t100\t100\t70\n6__fli__H2__X\t100\t100\t70\n7__stx1__stx1-a__X\t100\t100\t90\n8__stx2__stx2-d__X\t100\t100\t90\n9__stx2__stx2-a__X\t100\t100\t90\n10__eae__eae-42-5__X\t100\t100\t80\n11__ehxA__ehxA-7__X\t100\t100\t80", "-;H2", "stx1-a;stx2-a;stx2-d", "Positive", "Positive"),
    ("sample10", "1__adk__adk__X\t100\t100\t70\n2__fliC__H4__X\t100\t100\t70", "-;H4", "-", "-", "-"),
    ("sample11", "1__eae__eae-1__X\t100\t94\t70\n2__fliC__H6__X\t100\t100\t70", "-;H6", "-", "-", "-"),
    ("sample12", "1__stx1__stx1a__X\t100\t100\t80\n2__stx2__stx2c__X\t100\t100\t85\n3__fli__H21__X\t100\t100\t70", "-;H21", "stx1a;stx2c", "-", "-"),
]

for sample_name, res_content, expected_oh, expected_stx, expected_eae, expected_ehxA in test_cases:
    with TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        os.chdir(tmpdir)

        res_dir = tmpdir / f"examples/Results/{sample_name}/kma"
        res_dir.mkdir(parents=True)
        res_file = res_dir / f"{sample_name}.res"
        res_file.write_text("#Template\tTemplate_Coverage\tQuery_Identity\tDepth\n" + res_content)

        sheet = tmpdir / "samplesheet.tsv"
        sheet.write_text(
            "sample_name\tIllumina_read_files\tNanopore_read_file\tassembly_file\torganism\tvariant\tnotes\n"
            f"{sample_name}\tread1.fastq,read2.fastq\t-\t-\tEcoli\t-\t-\n"
        )

        results = EcoliResults.from_samplesheet(sheet)
        df = results.results_df
        row = df.iloc[0]
        
        # general output and functionality test
        assert row["sample_name"] == sample_name
        
        if row["OH"] != expected_oh:
            raise AssertionError(f"\nSample: {sample_name}\nExpected OH: {expected_oh}\nActual OH: {row['OH']}")
        assert row["OH"] == expected_oh
        
        if row["stx"] != expected_stx:
            raise AssertionError(f"\nSample: {sample_name}\nExpected stx: {expected_stx}\nActual stx: {row['stx']}")
        assert row["stx"] == expected_stx

        if row["eae"] != expected_eae:
            raise AssertionError(f"\nSample: {sample_name}\nExpected eae: {expected_eae}\nActual eae: {row['eae']}")
        assert row["eae"] == expected_eae

        if row["ehxA"] != expected_ehxA:
            raise AssertionError(f"\nSample: {sample_name}\nExpected ehxA: {expected_ehxA}\nActual ehxA: {row['ehxA']}")
        assert row["ehxA"] == expected_ehxA

        # sample specific information tests
        
        # without confliciting O and H typing, the OH column should be filled and the remaining four genes empty
        if sample_name == "sample1": 
            assert row["wzx"] == "-"
            assert row["wzy"] == "-"
            assert row["wzt"] == "-"
            assert row["wzm"] == "-"
        # with conflicts the OH should remain empty and the four 'conflicting' gene information remain filled
        elif sample_name == "sample6":
            assert row["wzx"] == "O157"
            assert row["wzy"] == "O157"
            assert row["wzt"] == "O8"
            assert row["wzm"] == "O8"
        elif sample_name == "sample10":
            assert row["Other"] == "adk"

print("All 12 syntehtic E. coli sample inline tests passed.")

Logging started for examples/Log/sample1_kma_fbi.log
Processing .res file: examples/Results/sample1/kma/sample1.res
Successfully processed sample: sample1
Logging started for examples/Log/sample2_kma_fbi.log
Processing .res file: examples/Results/sample2/kma/sample2.res
Successfully processed sample: sample2
Logging started for examples/Log/sample3_kma_fbi.log
Processing .res file: examples/Results/sample3/kma/sample3.res
Successfully processed sample: sample3
Logging started for examples/Log/sample4_kma_fbi.log
Processing .res file: examples/Results/sample4/kma/sample4.res
Successfully processed sample: sample4
Logging started for examples/Log/sample5_kma_fbi.log
Processing .res file: examples/Results/sample5/kma/sample5.res
Successfully processed sample: sample5
Logging started for examples/Log/sample6_kma_fbi.log
Processing .res file: examples/Results/sample6/kma/sample6.res
Successfully processed sample: sample6
Logging started for examples/Log/sample7_kma_fbi.log
Processing .res f

# Directive for ensuring that the code in your notebook get executed as a script

The code-block here below is required to ensure that the code in the notebook is also transferred to the module (script), otherwise it will just be a notebook. See [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details.

In [11]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()

ModuleNotFoundError: No module named 'nbdev'